### importing html

In [258]:
import pandas as pd
import numpy as np

html=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df=html[0]
print(df.head())
print(df.shape)

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
(287, 3)


In [259]:
## removing not assigned Borough
df=df[df["Borough"]!="Not assigned"]
## setting Borough as Neighbourhood for non assigned Neighbourhood
df.loc[df["Neighbourhood"]=="Not assigned","Neighbourhood"]=df.loc[df["Neighbourhood"]=="Not assigned","Borough"]
df.loc[df["Neighbourhood"]=="Not assigned",:]

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighbourhood


In [261]:
#grouping
x=df.groupby(["Postcode","Borough"]).count().reset_index()
#adding Neighbourhoods to grouping
for i in np.arange(x.shape[0]):
    temp=df[(df["Postcode"]==x.iloc[i,0]) & (df["Borough"]==x.iloc[i,1])]["Neighbourhood"].values.tolist()
    x.loc[i,"Neighbourhood"]=",".join(temp)

print(x[x["Postcode"]=="M4B"])
x.head()

   Postcode    Borough                   Neighbourhood
35      M4B  East York  Woodbine Gardens,Parkview Hill


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [262]:
#final shape
x.shape

(103, 3)

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    

In [263]:
## since geocoder doesnt work, csv is used

location=pd.read_csv("http://cocl.us/Geospatial_data")
location.head()
location.columns=["Postcode","Latitude","Longitude"]
x=pd.merge(x, location, on='Postcode', how='inner')
print(x.shape)
x.head()

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3

In [264]:
# Drawing Postcodes in Map
#!conda install -c conda-forge folium=0.5.0 --yes
from geopy.geocoders import Nominatim
import folium 
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude)
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood,postcode in zip(x['Latitude'], x['Longitude'], x['Borough'], x['Neighbourhood'],x['Postcode']):
    label = '{}, {}'.format(postcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada) 
map_canada

43.653963


In [266]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: KYU5RKHRHRWG4D4SCK1K2X22SGZFX5AJTGAAO3IEG3IC4Y4L
CLIENT_SECRET:FJ1HCXGLFETMD0IJKXJ1YVVCJJBT4QC30OHGAJY4IOO4ZOJU


(103, 5)

In [267]:
#Obtaining venues from foursquare

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans

LIMIT=100
def getNearbyVenues(names, latitudes, longitudes,postcodes,boroughs, radius=500):
    
    venues_list=[]
    for name, lat, lng,postcode,borough in zip(names, latitudes, longitudes,postcodes,boroughs):
        print(name+postcode)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(postcode,borough,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode','Borough','Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


toronto_venues = getNearbyVenues(names=x['Neighbourhood'],
                                   latitudes=x['Latitude'],
                                   longitudes=x['Longitude'],
                                    postcodes=x['Postcode'],
                                 boroughs=x['Borough']
                                  )

Rouge,MalvernM1B
Highland Creek,Rouge Hill,Port UnionM1C
Guildwood,Morningside,West HillM1E
WoburnM1G
CedarbraeM1H
Scarborough VillageM1J
East Birchmount Park,Ionview,Kennedy ParkM1K
Clairlea,Golden Mile,OakridgeM1L
Cliffcrest,Cliffside,Scarborough Village WestM1M
Birch Cliff,Cliffside WestM1N
Dorset Park,Scarborough Town Centre,Wexford HeightsM1P
Maryvale,WexfordM1R
AgincourtM1S
Clarks Corners,Sullivan,Tam O'ShanterM1T
Agincourt North,L'Amoreaux East,Milliken,Steeles EastM1V
L'Amoreaux WestM1W
Upper RougeM1X
Hillcrest VillageM2H
Fairview,Henry Farm,OrioleM2J
Bayview VillageM2K
Silver Hills,York MillsM2L
Newtonbrook,WillowdaleM2M
Willowdale SouthM2N
York Mills WestM2P
Willowdale WestM2R
ParkwoodsM3A
Don Mills NorthM3B
Flemingdon Park,Don Mills SouthM3C
Bathurst Manor,Downsview North,Wilson HeightsM3H
Northwood Park,York UniversityM3J
CFB Toronto,Downsview EastM3K
Downsview WestM3L
Downsview CentralM3M
Downsview NorthwestM3N
Victoria VillageM4A
Woodbine Gardens,Parkview HillM4B
Woodbine

In [154]:
toronto_venues.head()
toronto_venues[toronto_venues['Postcode']=="M9W"]
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
print(toronto_onehot.shape)
print(toronto_venues.shape)
# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = toronto_venues['Postcode']
toronto_onehot['Borough'] = toronto_venues['Borough'] 
toronto_onehot['Hoods'] = toronto_venues['Neighborhood']
toronto_onehot[toronto_onehot['Postcode']=="M9W"]
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


(2222, 266)
(2222, 9)


,Postcode,Borough,Hoods,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,Scarborough,"Rouge,Malvern",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,Scarborough,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,Scarborough,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [201]:
toronto_grouped = toronto_onehot.groupby(['Borough',"Postcode","Hoods"]).mean().reset_index()
print(toronto_grouped.shape)
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode','Borough','Hoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Hoods'] = toronto_grouped['Hoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 3:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
print(neighborhoods_venues_sorted.shape)

(99, 269)
(99, 6)


In [247]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering=toronto_grouped.drop(['Hoods','Postcode','Borough'], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 4, 4, 4, 1, 4, 4, 4, 4, 3], dtype=int32)

In [248]:
neighborhoods_venues_sorted['Cluster Labels']= kmeans.labels_

toronto_merged = x

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(['Hoods','Borough'], axis=1).set_index('Postcode'), on='Postcode', lsuffix='_left', rsuffix='_right',how="inner")

toronto_merged.head()



,Postcode,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,Cluster Labels
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,Fast Food Restaurant,Dessert Shop,Falafel Restaurant,2
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Bar,Golf Course,Yoga Studio,4
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Medical Center,Intersection,Spa,4
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Yoga Studio,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Caribbean Restaurant,Bakery,Fried Chicken Joint,4


In [229]:

toronto_merged.shape

(99, 9)

In [255]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster=int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [254]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,Cluster Labels
1,M1C,Bar,Golf Course,Yoga Studio,4
2,M1E,Medical Center,Intersection,Spa,4
4,M1H,Caribbean Restaurant,Bakery,Fried Chicken Joint,4
5,M1J,Convenience Store,Playground,Drugstore,4
7,M1L,Bakery,Bus Line,Park,4
8,M1M,Motel,American Restaurant,Skating Rink,4
9,M1N,College Stadium,Skating Rink,Café,4
10,M1P,Indian Restaurant,Pet Store,Vietnamese Restaurant,4
11,M1R,Breakfast Spot,Smoke Shop,Bakery,4
12,M1S,Lounge,Latin American Restaurant,Breakfast Spot,4
